In [6]:
import matplotlib.pyplot as pl
import numpy as np
from collections import Counter, OrderedDict
import random
import os
#Pour voir les matrices  en entiers sans troncature
import sys
import numpy
import time 
import datetime

numpy.set_printoptions(threshold=sys.maxsize)

###############################################################################
def check_name(path):
    #print(L)
    global seed_list, max_seed, nbre_images
    import re
    c=0
    N=os.listdir(path)
    #print(N)
    nbre_file=len(N) #V
    del(N)
    
    result=[0]*nbre_file
    seed_list=[0]*nbre_file

    B=(name for name in os.listdir(path))

    for c in range(nbre_file):
          
        A=next(B).split('_')[9]      
        #print(A)
    
        regex1 = re.compile('\d+')
        result[c]=re.findall(regex1,A)
        c+=1
    #print(result)
    for j in range(len(result)):
        #print(j)
        seed_list[j]=int(result[j][0])
        j+=1
        
    max_seed=max(seed_list)
    nbre_images=nbre_file/3
    
            
    #print(Z)
   
    #print(max_seed)
    
    return max_seed, seed_list, nbre_images

###############################################################################
def create_directory(path):
    import os
    #print(os.getcwd())

    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory failed, existing file with the same name : "+str(path))
    else:
        print ("Successfully created the directory : "+str(path))
    
    return
###############################################################################


def lattice_config(size,seed,p):
    global lattice, number_occupied
    
    print("lattice_config START", datetime.datetime.now())
    seed1=np.random.seed(seed)
    number_occupied=(size*size)*p
    #print(random, type(random))
    lattice=np.zeros((size,size), dtype=int)
    occupied=0
    #while len(list(zip(*lattice.nonzero())))<number_occupied:
    while occupied < number_occupied:
        #print(len(list(zip(*lattice.nonzero()))))
        i=random.randint(0,size-1)
        j=random.randint(0,size-1)
        if lattice[i,j]==0:
            lattice[i,j]=1
            occupied+=1
#     print(lattice)
    print("lattice_config END", datetime.datetime.now())
    return lattice, number_occupied

###############################################################################       
def color_point(cluster,top,side):  
     
 
    mid_diag=int(np.ceil(len(cluster/2)))-1
    quarter_diag=int(np.ceil(len(cluster)/3))-1  
    center=cluster[mid_diag][mid_diag]
    quarter=cluster[quarter_diag][quarter_diag]
    three_quarter=cluster[-(quarter_diag+1)][-(quarter_diag+1)]
    anti_quarter=cluster[(quarter_diag)][-(quarter_diag+1)]
    anti_three_quarter=cluster[-(quarter_diag+1)][quarter_diag]
    center_b=0
    quarter_b=0
    three_quarter_b=0
    anti_quarter_b=0
    anti_three_quarter_b=0  
    if top!=set():
        for i in range(len(top)):
            num_cluster=list(top)[i]
            if center==num_cluster:
                center_b=1
            if quarter_b==num_cluster:
                quarter_b=1
            
            if three_quarter==num_cluster:
                three_quarter_b=1
            
            if anti_quarter==num_cluster:
                anti_quarter_b=1
            
            if anti_three_quarter==num_cluster:
                anti_three_quarter_b=1

    if side!=set():
         for j in range(len(side)):
            num_cluster=list(side)[j]
            if center==num_cluster:
                center_b=1
            if quarter==num_cluster:
                quarter_b=1
            
            if three_quarter==num_cluster:
                three_quarter_b=1
            
            if anti_quarter==num_cluster:
                anti_quarter_b=1
            
            if anti_three_quarter==num_cluster:
                anti_three_quarter_b=1

    return quarter_b,three_quarter_b,anti_quarter_b,center_b,anti_three_quarter_b
    

        






def cluster_matrix(x, y, n_clusters, N, lattice,cluster, sizes):
    stack=[(x,y)]

    while len(stack)>0:

        x,y=stack.pop(-1)

        if lattice[x,y] == 1 and cluster[x,y]==-1:
            cluster[x,y] = n_clusters
            sizes[n_clusters] += 1     #augmente le nombre de site du cluster
            if y+1 < N:
                stack.append((x,y+1))

            if y-1 >= 0:

                stack.append((x,y-1))

            if x+1 < N:

                stack.append((x+1,y))

            if x-1 >= 0:
                stack.append((x-1,y))


################################################################

def cluster_matrix_pbc(x, y, n_clusters, N, lattice,cluster, sizes):
    stack=[(x,y)]
#     print('stack',stack, 'n_clusters',n_clusters)
#     print('-------')
    while len(stack)>0:
#         print('stack',stack)
        x,y=stack.pop(-1)
#         print('x','y',x,y,stack)
#         print('------------')
#         boo=(x==0)
#         lat1=(lattice[N-1,y]==1)
#         lat2=(lattice[x,N-1]==1)
#         print('boo',boo)
#         print('lat1',lat1,'lat2',lat2)
        if lattice[x%N,y%N] == 1 and cluster[x%N,y%N]==-1:
            cluster[x%N,y%N] = n_clusters
            sizes[n_clusters] += 1     #augmente le nombre de site du cluster

            stack.append((x,(y+1%N)))
#                 print('stack1',stack)
#             if y-1 >= 0:
#                 print('loop2')
            stack.append((x,(y-1)%N))
#                 print('stack2',stack)

#             if x+1 < N:
#                 print('loop3')
            stack.append(((x+1)%N,y))
#                 print('stack3',stack)
#             if x-1 >= 0:
#                 print('loop4')
            stack.append(((x-1)%N,y))
#         print(test)

###############################################################################
def size_spanning_cluster_old(spanning_set, cluster):
    global size_spanning
    size_spanning=0
    nnz1=cluster.nonzero()
    for k in range(len(spanning_set)):
        for i, j in zip(*nnz1):
            if cluster[i,j] ==list(spanning_set)[k]:
                size_spanning+=1
                
    return size_spanning
    
##############################################################################
def size_spanning_cluster(spanning_set,cluster,cluster_pbc_int):
    global size_spanning
    size=[]
    size_spanning=0
    span=list(spanning_set)
    n_cluster_pbc_int, counts = numpy.unique(cluster_pbc_int, return_counts=True)
    zip_lists=list(zip( n_cluster_pbc_int, counts))
    for k in range(len(spanning_set)):
        for i in range(len(zip_lists)):
            if zip_lists[i][0]==span[k]:
                size.append(zip_lists[i][1])
    if len(size)>1:
        size_spanning=max(size)
        
    return size_spanning

##############################################################################
def correlation_function_pbc_new_diagonal(lattice,l,n_clusters,seed):
    from math import sqrt
    
    
    
    

    
    start50=time.time()

    square_distance=np.zeros(l**2)
    corr2=np.zeros(l**2)
    corr_max_cluster=np.zeros(l**2)
    
    y_func=np.zeros(l**2)
    div_pbc=np.zeros(l**2)
    x_pbc_max_cluster=np.zeros(l**2)
    div_pbc_max_cluster=np.zeros(l**2)

    print('start correlation function',datetime.datetime.now())

    new_occupied = lattice.nonzero()
    new_occ=zip(*new_occupied)

    for y0 in range(l):
        for x0 in range(l):

            for y1 in range(y0,l):
                if y1==y0:

                    for x1 in range(x0,l):

                        x_distance=abs(x0-x1)
                        y_distance=abs(y0-y1)
                        
  

                        
                        if x_distance>(l/2):
                            x_distance=abs(l-x_distance) 
                        if y_distance>(l/2):
                            y_distance=abs(l-y_distance) 

                
                        distance=x_distance**2+ y_distance**2       
                        
                        div_pbc[distance]+=1
                        if x0!=x1 or y0!=y1:
                            div_pbc[distance]+=1
                    
                        if square_distance[distance]!=distance:
                            square_distance[distance]=distance
                        
                        
                        
                        if lattice[y0,x0]!=0 and lattice[y1,x1]!=0 and lattice[y0,x0]==lattice[y1,x1] : 
                            corr2[distance]+=1

                            if x0!=x1 or y0!=y1:
                                corr2[distance]+=1
                        if lattice[y0,x0]==n_clusters:
                            div_pbc_max_cluster[distance]+=1
                            if x0!=x1 or y0!=y1:
                                div_pbc_max_cluster[distance]+=1

                            if lattice[y0,x0]!=0 and  lattice[y1,x1]!=0 and lattice[y0,x0]==lattice[y1,x1]: 
                                corr_max_cluster[distance]+=1
                                if x0!=x1 or y0!=y1:
                                    corr_max_cluster[distance]+=1

                    
                else:
#                         print('loop y1!=y0')
                    for x1 in range(l): 
#                             print('x1',x1)
                        x_distance=abs(x0-x1)
                        y_distance=abs(y0-y1)
                        

                        if x_distance>(l/2):
                            x_distance=abs(l-x_distance)

                        
                        if y_distance>(l/2):
                            y_distance=abs(l-y_distance) 

                
                        distance= x_distance**2+ y_distance**2
#                     print('distance',distance)       
                        
                        div_pbc[distance]+=1
                        if x0!=x1 or y0!=y1:
                            div_pbc[distance]+=1
                                      
                        if square_distance[distance]!=distance:
                            square_distance[distance]=distance
#                                 print(square_distance)
                                      
                                    
                        if lattice[y0,x0]!=0 and lattice[y1,x1]!=0 and lattice[y0,x0]==lattice[y1,x1]: 
                            corr2[distance]+=1
#                                 print('corr2',corr2)
                            if x0!=x1 or y0!=y1:
                                corr2[distance]+=1
                        if lattice[y0,x0]==n_clusters:
                            div_pbc_max_cluster[distance]+=1
                            if x0!=x1 or y0!=y1:
                                div_pbc_max_cluster[distance]+=1
#                         print(div_pbc_max_cluster)
                            if lattice[y0,x0]!=0 and lattice[y1,x1]!=0 and lattice[y0,x0]==lattice[y1,x1]: 
                                corr_max_cluster[distance]+=1
                                if x0!=x1 or y0!=y1:
                                    corr_max_cluster[distance]+=1
#                             print(x_pbc_max_cluster)
        
    end50=time.time()-start50
    print('end correlation function',end50)
                        

    length=len(div_pbc)
                                      

    
    sqrt_distance=[sqrt(square_distance[h]) for h in range(len(square_distance))]
    sqrt_distance=np.array(sqrt_distance)
    print('len sqrt distance',len(sqrt_distance))
    #G.insert(0,0)
    
    average=[o/l for o,l in zip(corr2,div_pbc) ]
    average=np.array(average)
    index_max_corr=np.max(np.nonzero(corr2))
    corr3=corr2[:index_max_corr+1]
    index = np.where(corr3 == 0)[0]  #np.argwhere(np.isnan(W))
    correlation_value=np.delete(average,index)
    new_corr_before_average=np.delete(corr3,index)
    new_distance=np.delete(sqrt_distance,index)
    new_distance=np.delete(sqrt_distance,np.argwhere(sqrt_distance==0))
    
    new_distance=np.insert(new_distance, 0, 0)
    
#     print('new_distance',new_distance,len(new_distance))
    index_max=np.max(np.nonzero(new_corr_before_average))
    len_zero=len(new_distance)-(index_max+1)
    new_average=np.concatenate((correlation_value[:index_max+1],np.zeros(len_zero)))
    new_corr_before_average=np.concatenate((new_corr_before_average[:index_max+1],np.zeros(len_zero)))
    correlation_value=np.concatenate((correlation_value[:index_max+1],np.zeros(len_zero)))
    
    
    
    
    average_largest=[o/l for o,l in zip(corr_max_cluster,div_pbc) ]
    average_largest=np.array(average_largest)
    index_max_corr_largest=np.max(np.nonzero(corr_max_cluster))
    corr3_largest=corr_max_cluster[:index_max_corr_largest+1]
    
    index_largest = np.where(corr3_largest == 0)[0]  #np.argwhere(np.isnan(W))
    correlation_value_largest=np.delete(average_largest,index_largest)
    new_corr_before_average_largest=np.delete(corr3_largest,index_largest)
    new_distance_largest=np.delete(sqrt_distance,index_largest)
    new_distance_largest=np.delete(sqrt_distance,np.argwhere(sqrt_distance==0))
    
    new_distance_largest=np.insert(new_distance_largest, 0, 0)
    
#     print('new_distance',new_distance,len(new_distance))
    index_max_largest=np.max(np.nonzero(new_corr_before_average_largest))
    len_zero_largest=len(new_distance_largest)-(index_max_largest+1)
    new_average_largest=np.concatenate((correlation_value_largest[:index_max_largest+1],np.zeros(len_zero_largest)))
    new_corr_before_average_largest=np.concatenate((new_corr_before_average_largest[:index_max_largest+1],np.zeros(len_zero_largest)))
    correlation_value_largest=np.concatenate((correlation_value_largest[:index_max_largest+1],np.zeros(len_zero_largest)))
    
    

    
    end6=time.time()-start50
    print('end correlation function',end6)
    
    
    pl.plot(new_distance,new_corr_before_average,label='g(r) every clusters')
    pl.legend(loc='best')
    pl.xlabel('distance r')
    pl.ylabel('correlation function g(r)')
    pl.savefig('corr_before_average.png')
    pl.close()
    
   
    
    
   
    pl.plot(new_distance,new_average,label='g(r) every clusters')
    pl.legend(loc='best')
    pl.xlabel('distance r')
    pl.ylabel('correlation function g(r)')
    pl.savefig('corr_after_average_bis.png')
    pl.close()
    
    return new_distance,correlation_value, new_distance_largest,correlation_value_largest, new_corr_before_average
#################################################################################
def size_max(order_pbc,n_clusters):
    
    L=order_pbc
    values=[]
    keys=[]
#     print(L)
    for k, v in L.items():
        if k!=-1:
            keys.append(k)
            values.append(v)
    max_clus=max(values)
    max_n_clus=n_clusters
    
    return max_clus, max_n_clus


################################################################################
def mapping_cluster(cluster_nan,cluster_pbc_int,n_perco):
   
    mapping = dict(zip(cluster_nan.flat,cluster_pbc_int.flat))
    keys_mapping=mapping.keys()
    values_mapping=mapping.values()
    v=[p for p in values_mapping if p!=0]
    k=list(filter(lambda v: v==v, keys_mapping))
    zip_keys_values=list(zip(k,v))
    new_n=[]
    
    for i in range(0,len(zip_keys_values)):
        if (zip_keys_values[i][0] in n_perco) and (zip_keys_values[i][1] not in new_n):
            new_n.append(zip_keys_values[i][1])

    return new_n
    
    
###############################################################################                        
                        
def correlation_l(distance_to_site,Correlation_func, size_sys,proba_span,proba_largest,p):
    
    from math import sqrt 
    start=time.time()
    correlation=0
    sq_corr_l=0
    denom=0
    correlation_large=0
    denom_large=0
    for i in range(len(distance_to_site)):
        correlation+= ((distance_to_site[i]**2)*(Correlation_func[i]))#-proba_largest))
        correlation_large+= ((distance_to_site[i]**2)*(Correlation_func[i]-proba_largest))
        
                    
        denom+=(Correlation_func[i])#-proba_largest)
        denom_large+=(Correlation_func[i]-proba_largest)
        
    
    sq_corr_l=(correlation_large/(denom_large))
        
    
#         print('proba',proba_span)
#         print('corr',correlation,'denom', denom)
    if sq_corr_l==0:
        correlation_length=0
    elif sq_corr_l<0:
        correlation_length=0
#         print(i,correlation, denom)
    else:
#     print('corr',correlation,'denom', denom)
        correlation_length=sqrt(sq_corr_l)
    end=time.time()-start
    print('corre_length_end',end)
    return correlation_length
  

    
###############################################################################
def percolation(im,p,size_sys,seed):
    my_dpi=96 # DPI of the monitor

    import pickle
    import numpy as np
    np.random.seed(seed)
    random.seed(seed)
#     print('debut')

    #create_directory('percolating')   #if we want to classify them inside each density directory
    #create_directory('not_percolating')
    
    for t in range(im):
        print('--- working in image', t, datetime.datetime.now())
        start2=time.time()
        lattice_para=lattice_config(size_sys,seed,p)

        
        occupied = lattice_para[0].nonzero()
        n_clusters_pbc = 0
        n_clusters = 0
        cluster_pbc=np.zeros((size_sys,size_sys), dtype=int)-1
        cluster=np.zeros((size_sys,size_sys), dtype=int)-1
        sizes_pbc = Counter()
        sizes = Counter()
#         print('cluster')
        print("Hoshen-Kopelman PBS START", datetime.datetime.now())
        for i, j in zip(*occupied):
            if cluster_pbc[i,j] == -1:
                cluster_matrix_pbc(i, j, n_clusters_pbc, size_sys,lattice_para[0],cluster_pbc, sizes_pbc)
#                 cluster_matrix_pbc(i, j, n_clusters, size_sys,lattice,cluster, sizes)
                n_clusters_pbc += 1
        print("Hoshen-Kopelman PBC END", datetime.datetime.now())
        
        print("Hoshen-Kopelman HW START", datetime.datetime.now())
        for i, j in zip(*occupied):
            if cluster[i,j] == -1:
                cluster_matrix(i, j, n_clusters, size_sys,lattice_para[0],cluster, sizes)
#                 cluster_matrix_pbc(i, j, n_clusters, size_sys,lattice,cluster, sizes)
                n_clusters += 1
                #print(n_clusters)
        print("Hoshen-Kopelman HW END", datetime.datetime.now())
        
        print("PBC characterization", datetime.datetime.now())
            
        order_pbc=OrderedDict(sizes_pbc.most_common())
                  
        classification_pbc=list(order_pbc)
        #print(order)
        
        numbers_pbc = np.arange(0,n_clusters_pbc)
        weight_pbc=-np.sort(-(numbers_pbc))
        k_pbc=list(zip(classification_pbc,weight_pbc))
        
        correspondance_pbc=sorted(k_pbc, key = lambda t: t[0])
        unzip_pbc=list(zip(*correspondance_pbc))
        #print(unzip)
        new_mapping_pbc=unzip_pbc[1]
        
        
        print("HW characterization", datetime.datetime.now())
        
        order=OrderedDict(sizes.most_common())
                  
        classification=list(order)
        
        
        numbers = np.arange(0,n_clusters)
        weight=-np.sort(-(numbers))
        k=list(zip(classification,weight))
        
        correspondance=sorted(k, key = lambda t: t[0])
        unzip=list(zip(*correspondance))
        
        new_mapping=unzip[1]

        
        
        print("PBC coloring", datetime.datetime.now())
        
        cluster_pbc_int= np.array([new_mapping_pbc[v]+1 \
                            if not v == -1 else 0 for v in cluster_pbc.flat]).reshape(size_sys,size_sys)
        cluster_pbc_norm = np.array([(new_mapping_pbc[v]+1)/n_clusters_pbc \
                            if not v == -1 else 0 for v in cluster_pbc.flat]).reshape(size_sys,size_sys)
        
        
        print("HW coloring", datetime.datetime.now())
        
        cluster_int= np.array([new_mapping[v]+1 \
                            if not v == -1 else 0 for v in cluster.flat]).reshape(size_sys,size_sys)
        cluster_norm = np.array([(new_mapping[v]+1)/n_clusters \
                            if not v == -1 else 0 for v in cluster.flat]).reshape(size_sys,size_sys)
        cluster_nan = np.array([new_mapping[v]+1\
                           if not v == -1 else np.nan for v in cluster.flat]).reshape(size_sys,size_sys)

        print("correlation calculations", datetime.datetime.now())
        
        occ=len(list(zip(*cluster_int.nonzero())))
        start4=time.time()
        size_maxi=size_max(order_pbc,n_clusters)
        proba_largest= (size_maxi[0]/(size_sys**2))**2
        print('first correlation')
        corr_value=correlation_function_pbc_new_diagonal(cluster_pbc_int,size_sys,n_clusters_pbc,seed)
        print('end first correlation')
        
        square_proba=p*p
        #size_max(order_pbc,n_clusters)
        
        
        
        print('proba largest', proba_largest,'\n')
        print('occ',occ,'\n')
        print('max_clus',size_maxi[0],'\n')
        print(cluster_pbc_int,'\n')
        #
        
        pl.plot(corr_value[0],corr_value[1],label='g(r) every clusters')
        pl.plot(corr_value[2],corr_value[3], label='largest cluster')
        pl.axhline(y=square_proba, color='g', linestyle='--')
        pl.axhline(y=proba_largest, color='grey', linestyle='--')
        pl.legend(loc='best')
        pl.xlabel('distance r')
        pl.ylabel('correlation function g(r)')
        pl.title('Correlation function for system '+str(size_sys)+' at density '+str(p))
        pl.savefig('pc_________s'+str(seed)+'corr_func_'+'__p'+str(p)+'_L'+str(size_sys)+'_s'+str(seed)+'_'+'.png')
      
    
        corr_proba_largest=corr_value[1]-proba_largest
        corr_value_zipped=list(zip(corr_value[0],corr_value[1], corr_proba_largest,corr_value[2],corr_value[3]))
        
    
        header = '{0:^5s}   {1:^7s}   {2:^7s} {3:^5s}   {4:^7s} '.format('distance', 'Corr func','Corr func-proba largest','distance','Corr func largest')
        filename='pc_________s'+str(seed)+'Correlation_function_2.txt'
        np.savetxt(filename,corr_value_zipped, header=header, fmt=['    %.7f  ','    %.7f  ','    %.7f  ','  %.7f','  %.7f'])
        
        
#         final_correlation_func_pbc_m_max=[x - proba_largest for x in final_correlation_func_pbc]

        
#         corr3=list(zip(distance_to_site_div_pbc,Correlation_func_div_pbc_max,final_correlation_func_pbc_m_max))
#         header3 = '{0:^5s} {0:^1s}  {1:^7s}  '.format('distance','div', 'Correlation function - proba')
#         filename3='pc_________s'+str(seed)+'Correlation_function_m_largest_proba_'+str(proba_largest)+'.txt'
#         np.savetxt(filename3, corr3, header=header3, fmt=['    %.7f  ','    %.7f  ','  %.7f'])
        
#         pl.axhline(y=square_proba, color='g', linestyle='--')
#         pl.plot(distance_to_site_pbc,final_correlation_func_pbc,label='g(r) every clusters')
#         pl.plot(distance_to_site_pbc_max,final_correlation_func_pbc_max, label='g(r) largest cluster')
#         pl.plot(distance_to_site_pbc,final_correlation_func_pbc_m_max, label='g(r) every cluster-proba to belong to largest cluster')
#         pl.legend(loc='best')
#         pl.xlabel('distance r')
#         pl.ylabel('correlation function g(r)')
#         pl.title('Correlation function for system '+str(size_sys)+' at density '+str(p))
#         pl.savefig('pc_________s'+str(seed)+'corr_func_'+'__p'+str(p)+'_L'+str(size_sys)+'_s'+str(seed)+'_'+'.png')
        
#         pl.close()
        
#         pl.plot(distance_to_site_pbc,final_correlation_func_pbc,label='g(r) every clusters')
#         pl.plot(distance_to_site_pbc_max,final_correlation_func_pbc_max, label='g(r) largest cluster')
#         pl.plot(distance_to_site_pbc,final_correlation_func_pbc_m_max, label='g(r) every cluster-proba to belong to largest cluster')
#         pl.legend(loc='best')
#         pl.yscale('log')
#         pl.xlabel('distance r')
#         pl.ylabel('correlation function g(r)')
#         pl.title('Correlation function for system '+str(size_sys)+' at density '+str(p))
#         pl.savefig('pc_________s'+str(seed)+'log_corr_func_.png')
    
        
        
        end4=time.time()-start4
        
#         print('fin_corre', datetime.datetime.now())
        
        all_sizes = Counter(list(sizes.values()))
        
        #get size of largest cluster
        if n_clusters !=0:
            max_size = max(all_sizes.keys())
            
        if size_sys==100:
            size_im=131
        else:
            size_im=size_sys+ (size_sys/100)*30
       
        fig =pl.figure(figsize=((size_im/my_dpi), (size_im/my_dpi)), dpi=my_dpi)
        pl.axis('off')
        pl.imshow(cluster_pbc_norm,cmap='Greys')
#         print(cluster)
#         print(cluster2)
        occ=len(list(zip(*cluster_int.nonzero())))
#         occ2=len(list(zip(*cluster2.nonzero())))
#         print('occ1 bis',occ1_bis)
#         print('occ2',occ2)
        cmap2 = pl.cm.get_cmap('Greys')
#         occupied=len(list(zip(*cluster1.nonzero())))
       
        top=set(x for x in cluster_nan[0][:]).intersection(set(y for y in cluster_nan[-1][:]))
        side=set(w for w in cluster_nan[:,0]).intersection(set(z for z in cluster_nan[:,-1]))
        
        
        HWTB=0
        HWLR=0
        PBCTB=0
        PBCLR=0
         
        if (top!=set() or side!=set()):
            #os.chdir('percolating') 
            union_top_side=top.union(side)
            row1=cluster_nan[0][:]
            cluster_pseudo_pbc=np.row_stack((cluster_nan,row1))
            column1=cluster_pseudo_pbc[:,0]
            cluster_pseudo_pbc=np.column_stack((cluster_pseudo_pbc,column1))
            
            top_pbc=set(x for x in cluster_pseudo_pbc[0][:]).intersection(set(y for y in cluster_pseudo_pbc[-1][:]))
            side_pbc=set(w for w in cluster_pseudo_pbc[:,0]).intersection(set(z for z in cluster_pseudo_pbc[:,-1]))
            
            union_top_side_spanning_pbc= side_pbc.union(top_pbc)
            
            
            if (union_top_side_spanning_pbc!=set() and union_top_side_spanning_pbc.intersection(union_top_side)!=set):
                size_side_spanning_pbc=0
                size_top_spanning_pbc=0
                size_top_side_spanning=0
                
                if side_pbc!=set() and top_pbc!=set():                    
                    size_span_clus=size_spanning_cluster(union_top_side_spanning_pbc,cluster_pseudo_pbc,cluster_pbc_int)
                    size_top_side_spanning=size_span_clus
                    
                    rgba1=cmap2(next(iter(top_pbc)))
                    proba_span=(size_top_side_spanning/(size_sys*size_sys))
                    
                    PBCTB=1
                    PBCLR=1
                
                elif side_pbc!=set():
                    size_span_clus=size_spanning_cluster(side_pbc,cluster_pseudo_pbc,cluster_pbc_int)
                    size_side_spanning_pbc= size_span_clus
                    rgba2_pbc=cmap2(next(iter(side_pbc)))  #rgb color tuple + alpha
                    proba_span=(size_side_spanning/(size_sys*size_sys))**2
                    PBCLR=1
                 
                else:
                    size_span_clus=size_spanning_cluster(top_pbc,cluster_pseudo_pbc,cluster_pbc_int)
                    size_top_spanning_pbc=size_span_clus
                    rgba1_pbc=cmap2(next(iter(top_pbc)))
                    proba_span=(size_top_spanning/(size_sys*size_sys))
                    PBCTB=1
                
#                 mapping = dict(zip(cluster_nan.flat,cluster_pbc_int.flat))
#                 keys_mapping=mapping.keys()
#                 values_mapping=mapping.values()
                
                
#                 cluster=[]
#                 for i in:
#                     if l[i]==n:
#                         cluster.append(p[i])
#                         print(cluster)
    
             

           
#             color_point(cluster_int,top,side)  #,center_b=0,quarter_b=0,three_quarter_b=0,anti_quarter_b=0,anti_three_quarter_b=0)
            correlation_len=correlation_l(corr_value[0],corr_value[1], size_sys,proba_span,proba_largest,p) 
           
            
            
            
            
            
            
            size_side_spanning=0
            size_top_spanning=0
            rgba1=0
            rgba2=0
            
            if side!=set() and top!=set():
                size_span_clus_side=size_spanning_cluster(side,cluster_norm,cluster_pbc_int)
#                 size_side_spanning= size_spanning
                rgba2_pbc=cmap2(next(iter(side)))
                size_span_clus_top=size_spanning_cluster(top,cluster_norm,cluster_pbc_int)
                #size_top_spanning=size_spanning
                rgba1_pbc=cmap2(next(iter(top)))
                HWTB=1
                HWLR=1
                
            elif side!=set():
                size_span_clus_side=size_spanning_cluster(side,cluster_norm,cluster_pbc_int)
#                 size_side_spanning= size_spanning
                rgba2_pbc=cmap2(next(iter(side)))  #rgb color tuple + alpha
                HWLR=1
                 
            else:
                size_span_clus_top=size_spanning_cluster(top,cluster_norm,cluster_pbc_int)
#                 size_top_spanning=size_spanning
                rgba1_pbc=cmap2(next(iter(top)))
                HWTB=1
                
                
                
            filename1='pc_1_'+str(HWTB)+'_'+str(HWLR)+'_'+str(PBCTB)+'_'+str(PBCLR)+'__p'+str(p)+'_L'+str(size_sys)+'_s'+str(seed)+\
                        '_top_'+str(top)+'_side_'+str(side)+'_size_max_clus'+str(max_size)+\
                        '_occ_'+str(occ)+'_'
                       
            
            fig.savefig(filename1+'.png',bbox_inches='tight', pad_inches = 0,dpi=my_dpi)
            
#             filename='pc_________s'+str(seed)+'correlation'
            z=open(filename1+'corr_length'+'.txt', "w+")
            z.write('\n'+repr(correlation_len)+'\n')
            z.close()
                
                
#             f=open(filename1+'.txt', "w+")
#             f.write('Total number of cluster= '+ repr(n_clusters)+'\n')
#             f.write('Size of the largest cluster (number of site occupied)= '+ repr(max_size)+'\n')
#             f.write('Number of clusters with given size= ' +repr(sizes)+"\n")
#             f.write('Spanning cluster top-bottom = '+ repr(top)+' = '+repr(size_top_spanning)+"\n")
#             f.write('Spanning cluster side-side= '+ repr(side)+ ' = '+repr(size_side_spanning)+"\n")
#             f.write('color of the spanning cluster = '+repr(rgba1)+"\n")
#             f.write('color of the spanning cluster = '+repr(rgba2)+"\n")
#             f.close()
            
            

      
#             h= open(filename1+'.pkl', "wb")
#             pickle.dump(cluster1,h)
#             h.close()
            
            #os.chdir('..')
        else:
            size_max(order_pbc,n_clusters)
            #proba_max=(max_clus/(size_sys*size_sys))**2
            correlation_len=correlation_l(corr_value[0],corr_value[1], size_sys,proba_largest,proba_largest,p)
            
            #os.chdir('not_percolating')
            filename0='pc_0_'+str(HWTB)+'_'+str(HWLR)+'_'+str(PBCTB)+'_'+str(PBCLR)+'__p'+str(p)+'_L'+str(size_sys)+'_s'+str(seed)+\
                        '_size_max_clus'+str(max_clus)+'_n'+str(max_n_clus)+'_occ_'+str(occ)+'_'
            
            fig.savefig(filename0+'_.png', bbox_inches='tight',\
                        pad_inches = 0,dpi=my_dpi)
            
            #filename='pc_________s'+str(seed)+'correlation'
            z=open(filename0+'corr_length'+'.txt', "w+")
            z.write('\n'+repr(correlation_len)+'\n')
            z.close()
            
            
#             print(cluster_norm)
            
            
#             g=open(filename0+'.txt', "w+")
#             g.write('Total number of cluster= '+ repr(n_clusters)+'\n')
#             g.write('Size of the largest cluster (number of site occupied)= '+ repr(max_size)+'\n')
#             g.write('Sizes of each clusters (number associated to the cluster: number of occupied sites)= ' +repr(sizes)+"\n")
#             g.close()
            
#             i=open(filename0+'.pkl', "wb")
#             pickle.dump(cluster1,i)
#             i.close()
            #os.chdir('..')

        pl.close('all')
        seed+=1
        end2=time.time()-start2
        print(end2)

    return cluster,n_clusters,cluster_pbc,n_clusters_pbc, sizes_pbc,lattice,new_mapping,order,order_pbc,occ,\
sizes,cluster_int, top,side,cluster_nan,



#################################################################################################################################
def percolation_density(number_configs,perco_list,lattice_size,seed):
    global correlation, correlation_func
    import os
    #create_directory('L'+str(L))
    #os.chdir('L'+str(L))
    import time
    correlation=[]
    correlation_func=[]
    dens=[]
    start1= time.time()
    seed_ini=seed
    im_ini=number_configs
    for p in perco_list:
        new_im=0
        seed=seed_ini
        im=im_ini
        
        if os.path.exists('p'+str(p)) and len(os.listdir('p'+str(p)))!=0:
            print('A directory '+'p='+str(p)+' already exists')
            #print ("Creation of the directory failed")
            check_name('p'+str(p))
            print('A file already exist with max seed=',max_seed)
            os.chdir('p'+str(p))
            if im>= nbre_images:
                im=im-nbre_images
                while im > 0:
                    if seed in seed_list:
                        print('Image with seed = ',seed, 'already exists')
                        seed+=1
                    else:
                        dens.append(p)
                        percolation(1,p,lattice_size,seed) 
                        correlation.append(correlation_length)
#                         correlation_func.append(mean_int_func)
#                        im-=1
                        new_im+=1
                        seed+=1
                        print('NEW image with seed = ',seed, 'was created')
            else:
                print('The directory already contains ', nbre_images,\
                      ' images, please choose a higher number of configurations.')
                    
            os.chdir('..')
            if new_im!=0:
                print("-->",new_im, 'new images were created')
        
        else:
            create_directory('p'+str(p)+'new_diagonal')
            os.chdir('p'+str(p)+'new_diagonal')
            percolation(im,p,lattice_size,seed_ini)
            os.chdir('..')
            print(im, 'new images were created')
#     os.chdir('..')  
    end1=time.time()
    total_time=end1-start1
    print("Images generated in : ", total_time, "seconds")
    return


In [2]:
os.chdir('every_sites')

In [3]:
perco_list=[val/10000 for val in range(1000,10000,1000)]

In [7]:
os.getcwd()

'/home/physics/phrhmb/Desktop/test_5_01_21/every_sites'

In [1]:
percolation_density(1,perco_list,100,1)